In [2]:
import csv 
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [3]:
def createURL(loc,query):

    url_format ="https://{}.indeed.com/jobs?q={}&jt=internship&fromage=14"
    url = url_format.format(loc,query)
    return url

In [4]:
def autrepage(soup):
    sa = soup.find_all('div',{'class':'pagination'})
    
    a =True if sa !=[] else False
    return a
    


In [52]:
def getdfpays(loc,query):
    url = createURL(loc,query)
    

    
    liste = []
    entreprises=[]
    links = []
    salaries = []
    num = -1
    urlbase = url +"&start="
    start = 0
    while True:
        rep = requests.get(url)
        soup = BeautifulSoup(rep.text,'html.parser')
        #soup = soup.find("div",{'id':"mosaic-provider-jobcards"})
        cases = soup.find_all('a',{'class':"tapItem"})
        
        for u in cases:
            
            a= u.find('tr') 
            links.append(f"{url}&vjk={u['id'].split('_')[1]}")
            n = a.find_all('h2')

            other=u.find('span',{'class':'companyName'})
            
            company_=(other.get_text())
            
            entreprises.append(company_)
            if a.find('div',{'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}):
                salaries.append(a.find('div',{'class':'metadata salary-snippet-container'}).get_text())
        #                 print(a.find('div',{'class':'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly'}))
            else:
                salaries.append('Nope')
            for vb in n :
                h = vb.find_all('span')
            
                if(len(vb)==1):
                    liste.append(vb.get_text())
                    
                else:
                    liste.append(vb.get_text()[7:])
        if autrepage(soup) ==False:
            break
        start+=10
        url = urlbase + str(start)
                


    import pandas as pd
    df = pd.DataFrame()
    df["country"]=[loc for u in liste]
    df['titre']=liste
    df['entreprise']=entreprises
    df['salaire'] = salaries
    df['links'] = links
    
    return df


In [53]:
import pandas as pd
result  = pd.DataFrame()
pays = ['www','fr','de','uk','at','be','lu','ch','ie','nl','dk','ca','no','pt','es','fi','cn','jp','hu','ro']
l = []
for c in pays:
    try:
        df =getdfpays(c,'blockchain')
        
        result = result.append(df, ignore_index=True)
    except AttributeError : 
        print(c,': no result')




es : no result


Let's filter the results, I'm not interested in the jobs ads with keywords in the red_flags list 

In [61]:
red_flags = {'marketing','communication',"pré-embauche","management","legal","business"}  
green_flags = {'blockchain','solidity','react'}  

exporting to csv files

In [62]:


red = result['titre'].apply(lambda x  :set(x.lower().split()).intersection(red_flags)!=set())
green =result['titre'].apply(lambda x  :set(x.lower().split()).intersection(green_flags)!=set())
test = result

print(test.index)

test['green']= green
test['red']=red

test.to_csv("csv/alljobs.csv")

greens  = test[test['green']==True]
greens=greens.reset_index()
greens.to_csv("csv/green_flags.csv")

noreds = test[test['red']!=True]
noreds=noreds.reset_index()
noreds.to_csv("csv/filtered.csv")

reds  = test[test['red']==True]
reds=reds.reset_index()
reds.to_csv("csv/red_flags.csv")


RangeIndex(start=0, stop=101, step=1)


creation of an html file 

In [63]:

template = "<h2>title :<span>{}</span> pays : <span>{}</span> entreprise : <span>{}</span> <a href='{}'> link</a> </h2> \n"
a='<style> span{color : red;}</style>'
dict = noreds.to_dict()
for u in range(len(dict['country'])-1):
    a+=template.format(dict['titre'][u],dict['country'][u],dict['entreprise'][u],dict['links'][u])
with open('rendu.html', 'w') as f:
    f.write(a)